In [ ]:
 !pip install transformers
!pip install datasets

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead
  
tokenizer = AutoTokenizer.from_pretrained("t5-base")
  
model = AutoModelWithLMHead.from_pretrained("t5-base")

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:810: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
model.cuda()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

# Dataset

In [ ]:
from datasets import load_dataset
import torch
dataset = load_dataset("multi_nli",split="train[0:1000]")
batch_size = 1
dataloader_mnli = torch.utils.data.DataLoader(dataset, batch_size=batch_size,num_workers=2,pin_memory=True,shuffle=False)
mnli_length = len(dataset)
mnli_length

Using custom data configuration default


Dataset multi_nli downloaded and prepared to /root/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39. Subsequent calls will reuse this data.


1000

In [ ]:
from datasets import load_dataset
import torch
testset = load_dataset("multi_nli",split="validation_matched[:]")
batch_size = 64
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,shuffle=False)
test_length = len(testset)


Using custom data configuration default
Reusing dataset multi_nli (/root/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39)


In [ ]:
from datasets import load_dataset
import torch
testset = load_dataset("glue","rte",split="test")
batch_size = 64
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,shuffle=False)
test_length = len(testset)

Using the latest cached version of the module from /root/.cache/huggingface/modules/datasets_modules/datasets/glue/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad (last modified on Wed Jun  2 08:50:41 2021) since it couldn't be found locally at glue/glue.py or remotely (ConnectionError).
Reusing dataset glue (/root/.cache/huggingface/datasets/glue/rte/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [ ]:
from datasets import load_dataset
import torch
testset_b1 = load_dataset("multi_nli",split="validation_matched[:]")
batch_size = 1
testloader_b1 = torch.utils.data.DataLoader(testset_b1, batch_size=batch_size,shuffle=False)
test_length = len(testset)


Using custom data configuration default
Reusing dataset multi_nli (/root/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39)


# Next token predicition GPT style

In [ ]:
tokenizer.encode("contradiction")

[27252, 1]

In [ ]:
tokenizer.decode([21454])

'contradict'

In [ ]:
prompt0 = " hypothesis: "
prompt1 = " premise: "
index = 0
hypothesis = testset[index]["hypothesis"]
premise = testset[index]["premise"]

In [ ]:
import torch.nn.functional as F
input_ids = tokenizer.encode(prompt1+premise + prompt0 + hypothesis , return_tensors='pt').cuda()
logits = model(input_ids=input_ids,decoder_input_ids =torch.tensor([[0]]).cuda()).logits[:,-1,:]
F.softmax((torch.cat([logits[:,3], logits[:,7163],logits[:,27525]])),dim=-1)


tensor([2.1388e-01, 7.8612e-01, 1.1209e-11], device='cuda:0',
       grad_fn=<SoftmaxBackward>)

In [ ]:
min über argmax finden über testset

In [ ]:
[2.7681e-02, 9.7232e-01, 1.9821e-12]
[4.2925e-02, 9.5708e-01, 7.0674e-12]
[2.4699e-02, 9.7530e-01, 3.4003e-12]
[2.1388e-01, 7.8612e-01, 1.1209e-11]

In [ ]:
text = "A: All cats are cute. B: My cat is cute. -> entailment\n" \
       "A: Hans is walking. B: Hans is cute. -> neutral" \
       "A: Hans is walking. B: Hans is sleeping. -> contradiction" \
       "A: All men are mortal. Socrates is a man. B: Socrates is mortal -><extra_id_0> " 
input_ids = tokenizer.encode(text, return_tensors='pt').cuda()
out = model.generate(input_ids,num_beams=6)
tokenizer.decode(out[0])

'<pad><extra_id_0> contradictionA: All men are mortal.</s>'

In [ ]:
text = "A: All cats are cute. B: My cat is cute. -> entailment \n" \
       "A: Hans is walking. B: Hans is cute. -> neutral \n" \
       "A: Hans is walking. B: Hans is sleeping. -> contradiction \n" \
       "A: All men are mortal. Socrates is a man. B: Socrates is immortal -> <extra_id_0>" 
input_ids = tokenizer.encode(text, return_tensors='pt').cuda()
logits = model(input_ids=input_ids,decoder_input_ids =torch.tensor([[0,32099]]).cuda()).logits[:,-1,:]
tokenizer.decode(torch.argmax(logits))


'contradiction'

In [ ]:
tokenizer.encode("<extra_id_0> ->")

[32099, 3, 13114, 1]

In [ ]:
print(tokenizer.encode("entails"))
print(tokenizer.encode("is neutral"))
print(tokenizer.encode("contradicts"))

print(tokenizer.decode(3))
print(tokenizer.decode(19))
print(tokenizer.decode(21454))

[3, 35, 5756, 7, 1]
[21454, 7, 1]
[19, 7163, 1]

contradict
is


In [ ]:
from tqdm import tqdm_notebook
import numpy as np
import torch.nn.functional as F
num_right = 0
ent_count = 0
contr_count = 0
neutral_count = 0

ent_amount = 0
neutral_amount =0 
contr_amount = 0
c=0
with torch.no_grad():
  for i, mnli_instance in enumerate(tqdm_notebook(testloader)):

      text = "1. " + mnli_instance["hypothesis"][0] + " 2. " + mnli_instance["premise"][0] + "It is a fact, that statement 2 <extra_id_0> statement 1."
      input_ids = tokenizer.encode(text,return_tensors="pt").cuda()
      logits = (model(input_ids=input_ids,decoder_input_ids =torch.tensor([[0]]).cuda()).logits[:,-1,:])
      label = torch.argmax(F.softmax(torch.cat([logits[:,3]/1, logits[:,7163]/1,logits[:,27525]/1])))
      if torch.max(F.softmax(torch.cat([logits[:,3]/1, logits[:,7163]/1,logits[:,27525]/1]))) < 0.9:
        continue
      c+=1
      ent_count+=logits[:,3].item()
      neutral_count+=logits[:,19].item()
      contr_count+=logits[:,21454].item()
      if mnli_instance["label"]==0:
        ent_amount+=1
      if mnli_instance["label"]==1:
        neutral_amount+=1
      if mnli_instance["label"]==2:
        contr_amount+=1
      #if logits[:,28722] > 0 or logits[:,7163] > 0 or logits[:,27252] > 0:
      #  print("trulll")
      
      if label.cpu() == mnli_instance["label"]:
          num_right+=1

num_right/c

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0.3544574630667346

In [ ]:
print(ent_count)
print(neutral_count)
print(contr_count)

16.376203033141792
377.6361785978079
79.56536089879228


In [ ]:

#this makes only sense when using train + test with an unbalanced trainset
print(ent_count/ent_amount)
print(neutral_count/neutral_amount)
print(contr_count/contr_amount)

0.04802405581566508
1.1838124720934415
0.23401576734938906


In [ ]:
3
19
21454

using those tokens and text = "1. " + mnli_instance["hypothesis"][0] + " 2. " + mnli_instance["premise"][0] + "It is a fact, that statement 2 <extra_id_0> statement 1." has rand acc

# Prediction network

Use a fully connected nn which gets as input the probabilites for the tokens for one train instance. Output of the nn is 0/1/2. When given all the tokens, accurcy is random, when given only the label ""/neutral/contradiction acc goes up to 50% , still far below true acc, even thought nn would just have to pick the max.

problems:

*   may be undertrained
*   may be the wrong architecture
*   may just be to small



In [ ]:
tokenizer.vocab_size

32100

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(3, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 10)
        self.fc4 = nn.Linear(10,3)

        


    def forward(self, x):

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


net = Net()

In [ ]:
net.cuda()

Net(
  (fc1): Linear(in_features=3, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=3, bias=True)
)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [ ]:
torch.tensor([[[0]]*len(mnli_instance["premise"])]).squeeze(0).shape

torch.Size([1, 1])

In [ ]:
#train prediction network
from tqdm import tqdm_notebook
import random
import numpy as np
import copy
net.train()

for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, mnli_instance in enumerate(tqdm_notebook(dataloader_mnli)):


        texts = []
        for j in range(len(mnli_instance["premise"])):
          text = "hypothesis: " + mnli_instance["hypothesis"][j] + " premise: " + mnli_instance["premise"][j] 
          texts.append(text)


        input_ids = tokenizer(texts,return_tensors="pt",padding=True,truncation=True).input_ids.cuda()
        logits = F.softmax(model(input_ids=input_ids,decoder_input_ids =torch.tensor([[[0]]*len(mnli_instance["premise"])]).squeeze(0).cuda()).logits[:,-1,:])
        #print(logits[:,3].shape)
        inputs = torch.cat([logits[:,356].unsqueeze(1) ,logits[:,716].unsqueeze(1) ,logits[:,2752].unsqueeze(1) ],dim=-1)


        #inputs = torch.tensor([]).cuda()
        #for j in range(len(mnli_instance["premise"])):
        #  text = "hypothesis: " + mnli_instance["hypothesis"][j] + " premise: " + mnli_instance["premise"][j] 
        #  input_ids = tokenizer(text,return_tensors="pt").input_ids.cuda()
        #  logits = model(input_ids=input_ids,decoder_input_ids =torch.tensor([[0]]).cuda()).logits[:,-1,:]
        #  inputs = torch.cat([inputs,logits])


        
        # zero the parameter gradients
        optimizer.zero_grad()
        #print(inputs.shape)
        # forward + backward + optimize
        outputs = net(inputs)
        #print(outputs.shape)

        optimizer.zero_grad()

        loss = criterion(outputs,mnli_instance["label"].cuda())
        
        


        loss.backward()
        optimizer.step()

        


        if i % 10 == 9:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / (10)))
            running_loss = 0.0


        # print statistics
        running_loss += loss.item()

print('Finished Training')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


KeyboardInterrupt: ignored

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in tqdm_notebook(testloader):
        texts = []
        for j in range(len(mnli_instance["premise"])):
          text = "hypothesis: " + mnli_instance["hypothesis"][j] + " premise: " + mnli_instance["premise"][j] 
          texts.append(text)


        input_ids = tokenizer(texts,return_tensors="pt",padding=True,truncation=True).input_ids.cuda()
        logits = model(input_ids=input_ids,decoder_input_ids =torch.tensor([[[0]]*len(mnli_instance["premise"])]).squeeze(0).cuda()).logits[:,-1,:]
        inputs = torch.cat([logits[:,356].unsqueeze(1) ,logits[:,716].unsqueeze(1) ,logits[:,2752].unsqueeze(1) ],dim=-1)

        outputs = net(inputs)
        #print(outputs)

        _, predicted = torch.max(F.softmax(outputs.data), 1)
        #print(predicted)
        #break

        total += len(mnli_instance["premise"])
        correct += (predicted == mnli_instance["label"].cuda()).sum().item()



print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

# All combinations

In [ ]:
v = {"enc": [0 for x in range(0,32126)], "ecn": [0 for x in range(0,32126)], "nec": [0 for x in range(0,32126)], "nce": [0 for x in range(0,32126)], "cen": [0 for x in range(0,32126)], "cne": [0 for x in range(0,32126)], }

# Above baseline prediction

Same idea as the most biased token, but use the deviation from the baseline for each token to make a prediciton. This effectivly removes the need to select the specifcy label token. 

sanitiy check: acc with hypotheiss: label: stays the same

In [ ]:
#avg prob for each token, summed oder the trainset
from tqdm import tqdm_notebook
import numpy as np
import torch.nn.functional as F
sum = torch.tensor([0]*32128).cuda()
with torch.no_grad():
  for i, mnli_instance in enumerate(tqdm_notebook(testloader)):
    text = "mnli hypothesis: " + mnli_instance["hypothesis"][0] + " premise: " + mnli_instance["premise"][0]

    input_ids = tokenizer.encode(text,return_tensors="pt").cuda()
    logits = model(input_ids=input_ids,decoder_input_ids =torch.tensor([[0]]).cuda()).logits[:,-1,:]
    sum = sum + logits
avg  = sum / mnli_length
avg

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


tensor([[-274.7625, -181.4689, -194.0041,  ..., -555.9843, -557.7055,
         -558.6201]], device='cuda:0')

In [ ]:
len(sum[0])

32128

In [ ]:
a=[0 for x in range(0,180000)]

In [ ]:
print(avg[0,3])
print(avg[0,7163])
print(avg[0,27252])

tensor(-8.5739, device='cuda:0')
tensor(-7.2543, device='cuda:0')
tensor(-7.2496, device='cuda:0')


In [ ]:
#avg prob for each token for each label, summed over the trainset
ent = torch.tensor([0]*32128).cuda()
neutral = torch.tensor([0]*32128).cuda()
contr = torch.tensor([0]*32128).cuda()


ent_amount = 0
neutral_amount = 0
contr_amount = 0
with torch.no_grad():
  for i, mnli_instance in enumerate(tqdm_notebook(dataloader_mnli)):
    text = "mnli hypothesis: " + mnli_instance["hypothesis"][0] + " premise: " + mnli_instance["premise"][0]

    input_ids = tokenizer.encode(text,return_tensors="pt").cuda()
    logits = model(input_ids=input_ids,decoder_input_ids =torch.tensor([[0]]).cuda()).logits[:,-1,:]
    p = logits - avg
    print(p)
    if mnli_instance["label"]==0:
      ent = ent + p
      ent_amount+=1
    if mnli_instance["label"]==1:
      neutral = neutral + p
      neutral_amount+=1
    if mnli_instance["label"]==2:
      contr = contr + p
      contr_amount+=1
    break

print(ent/mnli_length)
print(neutral/mnli_length)
print(contr/mnli_length)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


tensor([[252.9452, 166.6922, 178.4959,  ..., 505.0792, 506.5644, 507.4203]],
       device='cuda:0')
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')
tensor([[0.2529, 0.1667, 0.1785,  ..., 0.5051, 0.5066, 0.5074]],
       device='cuda:0')
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')


In [ ]:
ent

tensor([[138.8237,  13.6456,  50.6006,  ..., 231.7615, 228.1238, 232.3025]],
       device='cuda:0')

In [ ]:
ent_amount

343

In [ ]:
#sorted tokens, the token that are most predictable for a label have higher values
#if a token is highly predictable for more than one label, that means that 

ent_val, ent_ind = torch.sort(ent/ent_amount,descending=True)
neutral_val, neutral_ind = torch.sort(neutral/neutral_amount,descending=True)
contr_val, contr_ind = torch.sort(contr/contr_amount,descending=True)

print(ent_ind[0,:10])
print(ent_val[0,:10])
print()

print(neutral_ind[0,:10])
print(neutral_val[0,:10])
print()

print(contr_ind[0,:10])
print(contr_val[0,:10])
print()

IndexError: ignored

In [ ]:
t = 7163
print((ent/ent_amount)[0,t])
print((neutral/neutral_amount)[0,t])
print((contr/contr_amount)[0,t])

NameError: ignored

In [ ]:
print(tokenizer.decode(ent_ind[0,:10]))
print(tokenizer.decode(neutral_ind[0,:10]))
print(tokenizer.decode(contr_ind[0,:10]))

upatteroyphp1/2tiptailUPpagehtml
Both Feedback Important Vezi Fair Wash Târgu Translat Rec Alb
Nici Nobody contradiction Nothing NoneNuNeither Nie AbsolutelyNo


In [ ]:
"""#most similiar vector
num_right = 0
with torch.no_grad():
  for i, mnli_instance in enumerate(tqdm_notebook(testloader)):
    text = ["" + mnli_instance["hypothesis"][j] + " " + mnli_instance["premise"][j] for j in range(0,len(mnli_instance["premise"]))]


    input = tokenizer(text,return_tensors="pt",padding=True)
    logits = model(input_ids=input["input_ids"].cuda(),attention_mask=input["attention_mask"].cuda(),decoder_input_ids =torch.tensor([[0]]*len(text)).cuda()).logits[:,-1,:]

    p = logits - avg.repeat(len(text),1) 

    e = torch.sum(torch.abs(p - ent/ent_amount),dim=-1)
    n = torch.sum(torch.abs(p - neutral/neutral_amount),dim=-1)
    c = torch.sum(torch.abs(p - contr/contr_amount),dim=-1)

    labels = torch.argmin(torch.cat([e.unsqueeze(0),n.unsqueeze(0),c.unsqueeze(0)]),dim=0)
    

    num_right+= torch.sum(labels.cpu() == mnli_instance["label"])


num_right/test_length"""

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


tensor(0.4567)

In [ ]:
torch.min(neutral/neutral_amount)

tensor(0.0970, device='cuda:0')

In [ ]:
#most similiar vector
num_right = 0
with torch.no_grad():
  for i, mnli_instance in enumerate(tqdm_notebook(testloader)):
    text = ["mnli hypothesis: " + mnli_instance["hypothesis"][j] + " premise: " + mnli_instance["premise"][j] for j in range(0,len(mnli_instance["premise"]))]


    input = tokenizer(text,return_tensors="pt",padding=True)
    logits = model(input_ids=input["input_ids"].cuda(),attention_mask=input["attention_mask"].cuda(),decoder_input_ids =torch.tensor([[0]]*len(text)).cuda()).logits[:,-1,:]

    p = logits - avg.repeat(len(text),1) 

    #e = (p * (ent/ent_amount)) * ((p * (ent/ent_amount)) >= 0) * (torch.sign(p))
    #n = (p * (neutral/neutral_amount)) * ((p * (neutral/neutral_amount)) >= 0) * (torch.sign(p))
    #c = (p * (contr/contr_amount)) * ((p * (contr/contr_amount)) >= 0) * (torch.sign(p))

    e = p * F.softmax((ent/ent_amount))
    n = p * F.softmax((neutral/neutral_amount))
    c = p * F.softmax((contr/contr_amount))


    e = torch.sum(e,dim=-1)
    n = torch.sum(n,dim=-1)
    c = torch.sum(c,dim=-1)





    labels = torch.argmax(torch.cat([e.unsqueeze(0),n.unsqueeze(0),c.unsqueeze(0)]),dim=0)
    

    num_right+= torch.sum(labels.cpu() == mnli_instance["label"])



num_right/test_length

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(0.3545)

In [ ]:
tensor(0.4613)

In [ ]:
 torch.tensor([4,-3]) *torch.tensor([[1,2],[6,9]])

tensor([[  4,  -6],
        [ 24, -27]])

In [ ]:
torch.sum(torch.tensor([[1,2,5],[2,3,5]]),dim=1)

tensor([ 8, 10])

In [ ]:
"""# use all tokens as a label, only slightly worse acc than normal for hypothesis: premise:
# for longer text random acc -> we made prediciton independant from label, which means that this analysis is a stronger argument in support that t5 cant solve mnli when given a qa format

num_right = 0
with torch.no_grad():
  for i, mnli_instance in enumerate(tqdm_notebook(testloader_b1)):
    text = "question: Does the premise entail, contradict or is neutral to the hypothesis? context: hypothesis: " + mnli_instance["hypothesis"][0] + " premise: " + mnli_instance["premise"][0]
    input_ids = tokenizer.encode(text,return_tensors="pt").cuda()
    logits = model(input_ids=input_ids,decoder_input_ids =torch.tensor([[0]]).cuda()).logits[:,-1,:]
    p = avg[:,:] - logits[:,:]
    e = torch.sum(p * F.softmax((ent/ent_amount)[:,:]))
    n = torch.sum(p * F.softmax((neutral/neutral_amount)[:,:]))
    c = torch.sum(p * F.softmax((contr/contr_amount)[:,:]))
    label = np.argmin([e,n,c])
    
    print
    if label == mnli_instance["label"]:
          num_right+=1
num_right/test_length"""

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


0.6737646459500765

In [ ]:
#using standart prompt and gpt-sytle(so text = "hypothesis: " + mnli_instance["hypothesis"][0] + "premise: " + mnli_instance["premise"][0] + "<extra_id_0>")
tensor([  413, 10206,  8170, 12786, 17637, 11537,  5756,  6880,  6492, 10500],
       device='cuda:0')
tensor([0.7507, 0.6906, 0.6891, 0.6862, 0.6862, 0.6848, 0.6789, 0.6774, 0.6760,
        0.6760], device='cuda:0')

tensor([ 2867, 23485, 16516, 26535,  4506, 12429, 27696, 30355,  7136, 12677],
       device='cuda:0')
tensor([0.6576, 0.6491, 0.6474, 0.6474, 0.6457, 0.6457, 0.6457, 0.6457, 0.6440,
        0.6440], device='cuda:0')

tensor([17167, 22009, 27252,  9428, 14794, 12924, 19685, 17584, 20510,  4168],
       device='cuda:0')
tensor([0.5869, 0.5677, 0.5650, 0.5445, 0.5445, 0.5349, 0.5349, 0.5116, 0.5062,
        0.5048], device='cuda:0')

#and acc of 54%, which means that nli capabilites are not even translaten when using the predict token
#entailment and neutral token are not even in the top10

#it was also not the case that i missed a space before extraid

#actually it just completly ignores the extra_id and just solves the normal task
# we could also investigate which tasks are stronger when we give t5 two tasks

In [ ]:
#text = "1. " + mnli_instance["hypothesis"][0] + " 2. " + mnli_instance["premise"][0] + "It is a fact, that statement 2 <extra_id_0> statement 1."
#gpt style
tensor([ 7641,  5978,  8193, 23990,  6407,  7229, 21655, 27724,  2396, 10286],
       device='cuda:0')
tensor([0.6833, 0.6686, 0.6598, 0.6598, 0.6569, 0.6540, 0.6540, 0.6510, 0.6422,
        0.6364], device='cuda:0')

tensor([ 8193, 30230,  4799, 21655,  5978, 23990, 14780, 30575, 13551, 25678],
       device='cuda:0')
tensor([0.7398, 0.7210, 0.6928, 0.6897, 0.6865, 0.6865, 0.6834, 0.6803, 0.6677,
        0.6646], device='cuda:0')

tensor([ 8193, 30230, 20215, 11572,  2396, 10286, 29096,  1178,  6407, 26999],
       device='cuda:0')
tensor([0.7000, 0.7000, 0.6941, 0.6794, 0.6765, 0.6765, 0.6765, 0.6706, 0.6706,
        0.6706], device='cuda:0')

und acc von 42%,aber mit labeln von testset

In [ ]:
# todo pro test instanz herausfinden, ob toekn größer oder kleiner dem basewert sind. Dann passend addieren. Zb wenn 3>avg, bekommt ent+1, contr+0.2,neutral+0.4 oder so. Dann max bilden

In [ ]:
tokenizer.decode([27252, 19218, 25632, 16715, 18616, 26682, 19685,  5839, 10942, 13373])

'contradiction delocintrarea nowherelipsawederNeither nonehardlyaucune'

In [ ]:
# ergebnis für hypothesis: premise: ,ohne softmax, mit denen bekommt man leicht über rand acc(40%)

tensor([    3,   587,  1120,  1216,  3919,  5266,  6328,  6418,  7102, 10316],
       device='cuda:0')
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0')

tensor([ 7163, 21323, 27744, 29156, 20998, 24623, 29389, 30913, 21010, 21228],
       device='cuda:0')
tensor([0.9898, 0.9830, 0.9830, 0.9813, 0.9796, 0.9796, 0.9796, 0.9761, 0.9744,
        0.9744], device='cuda:0')

tensor([27252, 19218, 25632, 16715, 18616, 26682, 19685,  5839, 10942, 13373],
       device='cuda:0')
tensor([0.4815, 0.4323, 0.4254, 0.4241, 0.4241, 0.4200, 0.4172, 0.4131, 0.4131,
        0.4131], device='cuda:0')


In [ ]:
# ergebnis für langen text,ohne softmax, mit denen bekommt man leicht über rand acc(40%)

tensor([ 4322, 11124, 21886,  9210, 20598, 25846, 27938, 21794,  9831, 18094],
       device='cuda:0')
tensor([0.7009, 0.6994, 0.6994, 0.6950, 0.6950, 0.6935, 0.6935, 0.6921, 0.6906,
        0.6906], device='cuda:0')

tensor([24714, 12991, 17069, 15148, 25946, 19384, 21021, 21233, 31817, 13516],
       device='cuda:0')
tensor([0.6048, 0.5911, 0.5843, 0.5826, 0.5809, 0.5792, 0.5775, 0.5775, 0.5775,
        0.5758], device='cuda:0')

tensor([15714,  6763, 24432, 21361, 12358,  7642, 29618,  9520, 13190, 20294],
       device='cuda:0')
tensor([0.5800, 0.5677, 0.5677, 0.5663, 0.5636, 0.5622, 0.5622, 0.5609, 0.5609,
        0.5609], device='cuda:0')

In [ ]:
#result for hypothesis: and premise:

tensor([ 1120,     3,     6,    31, 27441, 20734, 19197,    11,    87,  1216],
       device='cuda:0')
tensor([0.9355, 0.9311, 0.9179, 0.9164, 0.9150, 0.9106, 0.9062, 0.9032, 0.9003,
        0.8988], device='cuda:0')

tensor([ 7163,  7856, 21323, 29156,  1465, 12682, 20426, 21628, 26462, 22883],
       device='cuda:0')
tensor([0.9216, 0.6951, 0.6882, 0.6814, 0.6763, 0.6695, 0.6678, 0.6678, 0.6644,
        0.6610], device='cuda:0')

tensor([27252,  7598,  1327,   206, 29619, 26645, 28525, 12638, 19352, 20135],
       device='cuda:0')
tensor([0.9617, 0.5964, 0.5841, 0.5554, 0.5335, 0.5294, 0.5212, 0.5185, 0.5171,
        0.5171], device='cuda:0')

In [ ]:
from tqdm import tqdm_notebook
import numpy as np
import torch.nn.functional as F
num_right = 0
ent_count = 0
contr_count = 0
neutral_count = 0

ent_amount = 0
neutral_amount =0 
contr_amount = 0
with torch.no_grad():
  for i, mnli_instance in enumerate(tqdm_notebook(dataloader_mnli)):

      text = "question: Which option is right? context: statement 1:" + mnli_instance["hypothesis"][0] + " statement 2: " + mnli_instance["premise"][0] + "Option 1: statement 2 entails statement 1. Option 2: statement 2 is neutral to statement 1. Option 3: statement 2 contradicts statement 1."

      input_ids = tokenizer.encode(text,return_tensors="pt").cuda()
      logits = F.softmax(model(input_ids=input_ids,decoder_input_ids =torch.tensor([[0]]).cuda()).logits[:,-1,:])
      label = torch.argmax(torch.cat([logits[:,4322]/1.3330772191637655e-05, logits[:,24714]/5.47142535705647e-05,logits[:,15714]/2.9161071427877272e-05]))
      ent_count+=logits[:,4322].item()
      neutral_count+=logits[:,24714].item()
      contr_count+=logits[:,15714].item()
      if mnli_instance["label"]==0:
        ent_amount+=1
      if mnli_instance["label"]==1:
        neutral_amount+=1
      if mnli_instance["label"]==2:
        contr_amount+=1
      #if logits[:,28722] > 0 or logits[:,7163] > 0 or logits[:,27252] > 0:
      #  print("trulll")
      
      if label.cpu() == mnli_instance["label"]:
          num_right+=1

num_right/mnli_length

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0.397

In [ ]:
print(ent_count/ent_amount)#das darf man eig garnicht machen
print(neutral_count/neutral_amount)
print(contr_count/contr_amount)

1.9546586791257558e-08
9.320997201118347e-08
3.989202657712349e-08


In [ ]:
print(ent_count)#das darf man eig garnicht machen
print(neutral_count)
print(contr_count)

1.3330772191637655e-05
5.47142535705647e-05
2.9161071427877272e-05


In [ ]:
mit 18682/17011/13745 bekommt man für       text = "question: Which option is right? context: statement 1:" + mnli_instance["hypothesis"][0] + " statement 2: " + mnli_instance["premise"][0] + "Option 1: statement 2 entails statement 1. Option 2: statement 2 is neutral to statement 1. Option 3: statement 2 contradicts statement 1."
42%, also leicht besser als rand

# most biased tokens

question: which labels should we use for ent/neutral/contr without trying all the 50000^3 possibilites


look for labels that have the highest deviation from their baseline for one label. So we first calculate the basline for each token, by computing the mean over all the training instances. Then we calculate the baseline for each token for each label. Then divide the label specific baseline by the baseline. If there is a token that is highly predictable for a label it will have a high value. Sort them

In [ ]:
torch.tensor([1,2]) / torch.tensor([5,6])

tensor([0.2000, 0.3333])

In [ ]:
from tqdm import tqdm_notebook
import numpy as np
import torch.nn.functional as F
sum_ent = torch.tensor([0]*32128).cuda()
sum_neutral = torch.tensor([0]*32128).cuda()
sum_contr = torch.tensor([0]*32128).cuda()

ent_amount = 0
neutral_amount = 0
contr_amount = 0


with torch.no_grad():
  sum=torch.tensor([0]*32128).cuda()
  for i, mnli_instance in enumerate(tqdm_notebook(dataloader_mnli)):
    text = "1. " + mnli_instance["hypothesis"][0] + " 2. " + mnli_instance["premise"][0] + "It is a fact, that statement 2 <extra_id_0> statement 1."
    input_ids = tokenizer.encode(text,return_tensors="pt").cuda()
    logits = F.softmax(model(input_ids=input_ids,decoder_input_ids =torch.tensor([[0]]).cuda()).logits[:,-1,:])
    if mnli_instance["label"]==0:
      sum_ent = sum_ent + logits
      ent_amount+=1

    if mnli_instance["label"]==1:
      sum_neutral = sum_neutral + logits
      neutral_amount+=1

    if mnli_instance["label"]==2:
      sum_contr = sum_contr + logits
      contr_amount+=1


sum_ent = sum_ent / ent_amount
sum_neutral = sum_neutral / neutral_amount
sum_contr = sum_contr / contr_amount

sum = sum_ent + sum_neutral + sum_contr

sum_ent = sum_ent / sum
sum_neutral = sum_neutral / sum
sum_contr = sum_contr / sum

print(sum_ent)
print(sum_neutral)
print(sum_contr)

      




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



tensor([[0.5346, 0.6291, 0.6537,  ..., 0.2485, 0.2452, 0.2459]],
       device='cuda:0')
tensor([[0.3522, 0.2466, 0.2556,  ..., 0.4662, 0.4665, 0.4653]],
       device='cuda:0')
tensor([[0.1132, 0.1243, 0.0908,  ..., 0.2853, 0.2883, 0.2888]],
       device='cuda:0')


In [ ]:
ent_val, ent_ind = torch.sort(sum_ent,descending=True)
neutral_val, neutral_ind = torch.sort(sum_neutral,descending=True)
contr_val, contr_ind = torch.sort(sum_contr,descending=True)

print(ent_ind[0,:10])
print(ent_val[0,:10])
print()

print(neutral_ind[0,:10])
print(neutral_val[0,:10])
print()

print(contr_ind[0,:10])
print(contr_val[0,:10])
print()

tensor([    3, 19197, 20734, 14371, 22508, 24161,  5266, 10998, 13943, 27441],
       device='cuda:0')
tensor([0.8992, 0.8241, 0.8202, 0.8034, 0.7889, 0.7781, 0.7771, 0.7741, 0.7741,
        0.7701], device='cuda:0')

tensor([29048, 16239, 25052, 18566, 18026,  1960, 30598,  7163, 20092,   469],
       device='cuda:0')
tensor([0.9812, 0.9587, 0.8740, 0.8736, 0.8713, 0.8358, 0.8355, 0.8302, 0.8083,
        0.8011], device='cuda:0')

tensor([18616, 19218, 13373, 28369, 14607, 15850, 20864, 27826, 30620, 31527],
       device='cuda:0')
tensor([0.9924, 0.9879, 0.9847, 0.9823, 0.9810, 0.9807, 0.9573, 0.9488, 0.9438,
        0.9426], device='cuda:0')



In [ ]:
tokenizer.decode([  125, 14410,  9170, 20561, 17849,  8178, 21747,  6104, 11835,  4479])

'what passivewhat proactivewelcher Jon cadouquelle Baiquel'

In [ ]:
# hypothesis: and premise:, train[2000:4000]
tensor([    3, 19197, 20734, 14371, 22508, 24161,  5266, 10998, 13943, 27441],
       device='cuda:0')
tensor([0.8992, 0.8241, 0.8202, 0.8034, 0.7889, 0.7781, 0.7771, 0.7741, 0.7741,
        0.7701], device='cuda:0')

tensor([29048, 16239, 25052, 18566, 18026,  1960, 30598,  7163, 20092,   469],
       device='cuda:0')
tensor([0.9812, 0.9587, 0.8740, 0.8736, 0.8713, 0.8358, 0.8355, 0.8302, 0.8083,
        0.8011], device='cuda:0')

tensor([18616, 19218, 13373, 28369, 14607, 15850, 20864, 27826, 30620, 31527],
       device='cuda:0')
tensor([0.9924, 0.9879, 0.9847, 0.9823, 0.9810, 0.9807, 0.9573, 0.9488, 0.9438,
        0.9426], device='cuda:0')

In [ ]:
# hypothesis: and premise:, train[0:2000]
tensor([    3,   837, 20734,   412, 19197, 14371, 10998,  5266, 22508, 27441],
       device='cuda:0')
tensor([0.8970, 0.8494, 0.8279, 0.8222, 0.8212, 0.7910, 0.7894, 0.7871, 0.7845,
        0.7795], device='cuda:0')

tensor([ 7163, 31510, 20998, 23480, 24788, 24750, 29156, 31120, 21323, 31178],
       device='cuda:0')
tensor([0.8462, 0.8138, 0.7635, 0.7626, 0.7361, 0.7284, 0.7193, 0.7187, 0.7136,
        0.7123], device='cuda:0')

tensor([18616, 19218, 28369, 14607, 13373, 15850, 20864, 27826, 30620,  7362],
       device='cuda:0')
tensor([0.9909, 0.9873, 0.9831, 0.9812, 0.9800, 0.9773, 0.9745, 0.9538, 0.9513,
        0.9449], device='cuda:0')


In [ ]:
#results for  "question: Which option is right? context: statement 1:" + mnli_instance["hypothesis"][0] + " statement 2: " + mnli_instance["premise"][0] + "Option 1: statement 2 entails statement 1. Option 2: statement 2 is neutral to statement 1. Option 3: statement 2 contradicts statement 1."
tensor([19197, 23869, 31946,  3804, 29121, 28149, 31055,  8545, 16545, 31447],
       device='cuda:0')
tensor([0.7818, 0.7526, 0.6689, 0.6672, 0.6234, 0.6044, 0.6009, 0.6003, 0.5872,
        0.5842], device='cuda:0')

tensor([  125, 14410,  9170, 20561, 17849,  8178, 21747,  6104, 11835,  4479],
       device='cuda:0')
tensor([0.8829, 0.8059, 0.7840, 0.7270, 0.7155, 0.6883, 0.6488, 0.6344, 0.6248,
        0.6115], device='cuda:0')

tensor([13643,  3644, 11122, 12084,  4281,  1980,  7078, 20592,  1845, 14256],
       device='cuda:0')
tensor([0.9996, 0.9942, 0.9742, 0.9735, 0.9713, 0.9632, 0.9599, 0.9489, 0.9478,
        0.9426], device='cuda:0')

In [ ]:
# rersult for hypothesis: and premise:

tensor([    3,   837, 20734, 19197,   412, 10998, 14371, 22508,  5266, 27441],
       device='cuda:0')
tensor([0.8954, 0.8245, 0.8213, 0.8035, 0.7849, 0.7841, 0.7839, 0.7754, 0.7752,
        0.7711], device='cuda:0')

tensor([ 7163, 31510, 23480, 20998, 24788, 24750, 29156, 30913, 21323, 31178],
       device='cuda:0')
tensor([0.8421, 0.8205, 0.7517, 0.7464, 0.7190, 0.7179, 0.7167, 0.7037, 0.7025,
        0.7017], device='cuda:0')

tensor([18616, 19218, 14607, 28369, 13373, 15850, 20864, 27826, 30620,  7362],
       device='cuda:0')
tensor([0.9898, 0.9870, 0.9808, 0.9801, 0.9771, 0.9762, 0.9728, 0.9526, 0.9467,
        0.9428], device='cuda:0')

'lipsa deloc nimeniabsoluaucune niciunsadlypathetictermeni nimic'

In [ ]:
from tqdm import tqdm_notebook
import numpy as np
import torch.nn.functional as F
num_right = 0
ent_count = 0
contr_count = 0
neutral_count = 0

ent_amount = 0
neutral_amount =0 
contr_amount = 0
with torch.no_grad():
  for i, mnli_instance in enumerate(tqdm_notebook(testloader)):

      text = "hypothesis: " + mnli_instance["hypothesis"][0] + " premise: " + mnli_instance["premise"][0] 
      input_ids = tokenizer.encode(text,return_tensors="pt").cuda()
      logits = F.softmax(model(input_ids=input_ids,decoder_input_ids =torch.tensor([[0]]).cuda()).logits[:,-1,:])
      label = torch.argmax(torch.cat([logits[:,1120]/2.834065510211327e-07, logits[:,7163]/1.035108953294475,logits[:,27252]/1.0598568164649003]))
      ent_count+=logits[:,1120].item()
      neutral_count+=logits[:,7163].item()
      contr_count+=logits[:,27252].item()
      if mnli_instance["label"]==0:
        ent_amount+=1
      if mnli_instance["label"]==1:
        neutral_amount+=1
      if mnli_instance["label"]==2:
        contr_amount+=1
      #if logits[:,28722] > 0 or logits[:,7163] > 0 or logits[:,27252] > 0:
      #  print("trulll")
      
      if label.cpu() == mnli_instance["label"]:
          num_right+=1

num_right/test_length 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0.855

In [ ]:
token 1120 funktioniert besser als token 3 für ent, aber nur 0.002 mehr

In [ ]:
tokenizer.decode(1120)

'app'

In [ ]:
print(ent_count/ent_amount)
print(neutral_count/neutral_amount)
print(contr_count/contr_amount)

2.834065510211327e-07
1.035108953294475
1.0598568164649003


In [ ]:
print(ent_count)
print(neutral_count)
print(contr_count)

9.664163389820624e-05
330.1997561009375
360.3513175980661


In [ ]:
# prediction nicht darauf basieren, welcher token höhere normalisierte wkeit hat, sondern ob er in einer instanz höher oder niedriger wie der durchschnittswert von diesem token ist

#TODO statt summe, bei jeder trainingsinstanz festhalten, ob man mit diesem token richtig predicten würde, wenn ja eins erhöhen

# optimal token

sum over the softmax of the outputs for every label. Token that have the highest value have the highest predictability for a class.
Problems:
- Can be skewed by outliers
- Does not imply a scheme to decide which label to choose at testime automatically, but it can be used for normal prediction/ensembles

In [ ]:
from tqdm import tqdm_notebook
import numpy as np
import torch.nn.functional as F
c=0
with torch.no_grad():
  sum=torch.tensor([0]*32128).cuda()
  for i, mnli_instance in enumerate(tqdm_notebook(testloader)):
      if not mnli_instance["label"]==2:
        continue
      c+=1
      text = "1. " + mnli_instance["hypothesis"][0] + " 2. " + mnli_instance["premise"][0] + "It is a fact, that statement 2 <extra_id_0> statement 1."
      input_ids = tokenizer.encode(text,return_tensors="pt").cuda()
      logits = F.softmax(model(input_ids=input_ids,decoder_input_ids =torch.tensor([[0]]).cuda()).logits[:,-1,:])
      sum = sum + logits

c

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


738

In [ ]:
val,ind = torch.sort(sum,descending=True)
val[0,:30]

tensor([6.9360e+02, 1.3399e+01, 8.5308e+00, 4.6559e+00, 2.4859e+00, 1.4106e+00,
        1.3261e+00, 1.1863e+00, 1.1552e+00, 9.1381e-01, 8.9107e-01, 6.6692e-01,
        4.4322e-01, 3.7178e-01, 3.4229e-01, 3.1519e-01, 2.8108e-01, 2.5090e-01,
        1.3439e-01, 1.0023e-01, 8.6486e-02, 8.3883e-02, 8.2752e-02, 8.1493e-02,
        8.0539e-02, 7.1429e-02, 6.8011e-02, 6.7271e-02, 6.5737e-02, 6.0895e-02],
       device='cuda:0')

In [ ]:
tokenizer.decode([10231,  1182,  2493,     3,    59, 16836,   209, 10998, 13745, 17011,
         1160,   204, 10747,   933,   220, 13661, 20592, 13643,  1877,   931,
         9546,  1980,  3025,     8,   314,  6864,  1845, 19197,  1682,    96])

'Option option statement  not Statement 1 True Choice Options choice 2 Fal Not 3 Alternative Accept accepting 3. optionsOpti Ad Form the 4 2.0 accept duplicate 2. "'

## Results

In [ ]:
# tokens, that have the hightest predictability for contradiction , using the prompt "question: Which option is right? context: statement 1:" + mnli_instance["hypothesis"][0] + " statement 2: " + mnli_instance["premise"][0] + "Option 1: statement 2 entails statement 1. Option 2: statement 2 is neutral to statement 1. Option 3: statement 2 contradicts statement 1."

[6.9360e+02, 1.3399e+01, 8.5308e+00, 4.6559e+00, 2.4859e+00, 1.4106e+00,
        1.3261e+00, 1.1863e+00, 1.1552e+00, 9.1381e-01, 8.9107e-01, 6.6692e-01,
        4.4322e-01, 3.7178e-01, 3.4229e-01, 3.1519e-01, 2.8108e-01, 2.5090e-01,
        1.3439e-01, 1.0023e-01, 8.6486e-02, 8.3883e-02, 8.2752e-02, 8.1493e-02,
        8.0539e-02, 7.1429e-02, 6.8011e-02, 6.7271e-02, 6.5737e-02, 6.0895e-02]

[10231,  1182,  2493,     3,    59, 16836,   209, 10998, 13745, 17011,
         1160,   204, 10747,   933,   220, 13661, 20592, 13643,  1877,   931,
         9546,  1980,  3025,     8,   314,  6864,  1845, 19197,  1682,    96]

Option option statement  not Statement 1 True Choice Options choice 2 Fal Not 3 Alternative Accept accepting 3. optionsOpti Ad Form the 4 2.0 accept duplicate 2. "

In [ ]:
# tokens, that have the hightest predictability for neutral , using the prompt "question: Which option is right? context: statement 1:" + mnli_instance["hypothesis"][0] + " statement 2: " + mnli_instance["premise"][0] + "Option 1: statement 2 entails statement 1. Option 2: statement 2 is neutral to statement 1. Option 3: statement 2 contradicts statement 1."

[5.4008e+02, 9.7588e+00, 5.4749e+00, 3.6816e+00, 2.4181e+00, 1.1448e+00,
        1.0622e+00, 9.9868e-01, 9.0435e-01, 7.3559e-01, 6.0602e-01, 5.3170e-01,
        4.2433e-01, 3.3210e-01, 2.9439e-01, 2.5050e-01, 1.2508e-01, 7.7386e-02,
        6.8835e-02, 6.8521e-02, 6.5546e-02, 6.4109e-02, 6.1281e-02, 5.4061e-02,
        4.9999e-02, 4.6036e-02, 4.3535e-02, 4.2350e-02, 3.9841e-02, 3.9697e-02]

[10231,  1182,  2493,     3,    59, 10998,   209, 16836, 13745, 17011,
         1160,   204, 10747,   933,   220, 13661,  1877,   931,  3025,  9546,
          314,  6864,     8,  1682, 19197,    96,  5150,  4495,   505,  1300]

Option option statement  not True 1 Statement Choice Options choice 2 Fal Not 3 Alternative 3. options FormOpti 4 2.0 the 2. duplicate " Standard Op 8 1.

In [ ]:
# tokens, that have the hightest predictability for entailement , using the prompt "question: Which option is right? context: statement 1:" + mnli_instance["hypothesis"][0] + " statement 2: " + mnli_instance["premise"][0] + "Option 1: statement 2 entails statement 1. Option 2: statement 2 is neutral to statement 1. Option 3: statement 2 contradicts statement 1."

tensor([6.3083e+02, 1.5783e+01, 1.1896e+01, 6.4135e+00, 5.9178e+00, 2.1455e+00,
        1.6830e+00, 1.5822e+00, 1.1003e+00, 1.0337e+00, 8.5550e-01, 8.3769e-01,
        5.5800e-01, 5.0037e-01, 4.4049e-01, 4.0003e-01, 3.1413e-01, 2.0547e-01,
        1.1852e-01, 1.1674e-01, 9.6514e-02, 9.6447e-02, 9.6174e-02, 9.4598e-02,
        7.9170e-02, 7.9038e-02, 6.8018e-02, 6.6029e-02, 5.9195e-02, 5.8820e-02],
[10231,  1182,  2493,     3,    59,   209, 16836, 10998, 13745,  1160,
        17011,   204, 10747,   933, 19197,   220, 13661,  1877,  6864,   931,
         3025,     8,  1682,   314,  9546,  1300,    96,  8613,  4495,  5150]
       
Option option statement  not 1 Statement True Choice choice Options 2 Fal Not duplicate 3 Alternative 3. 2.0 options Form the 2. 4Opti 1. " 1.5 Op Standard

In [ ]:
# tokens, that have the hightest predictability for contradiction , using the prompt hypothesis: and premise:
tensor([3.3714e+03, 2.4327e+02, 7.0882e+01, 9.6478e-02, 2.6534e-02, 1.8580e-02,
        8.5092e-03, 7.7647e-03, 6.2826e-03, 6.2697e-03, 4.5502e-03, 4.2940e-03,
        4.1223e-03, 3.8738e-03, 3.6271e-03, 3.6002e-03, 3.4849e-03, 3.3956e-03,
        3.0399e-03, 3.0029e-03, 2.9057e-03, 2.8247e-03, 2.7409e-03, 2.4622e-03,
        2.3603e-03, 2.3394e-03, 2.3237e-03, 2.2855e-03, 2.2481e-03, 2.1406e-03],
       device='cuda:0')

tensor([27252,  7163,     3,    59, 10747,   206,   150,     6,  2841,  4486,
           96,  1877,   673,    16,  1465,   105,     5,  6864,    29, 10998,
        32095,    18,     1,    20, 32097,   209, 32081, 32093,   220,    41],

In [ ]:
# tokens, that have the hightest predictability for neutral , using the prompt hypothesis: and premise:
tensor([2.4199e+03, 2.5138e+02, 2.3008e+02, 8.5957e-02, 1.5929e-02, 1.4471e-02,
        9.1645e-03, 8.6295e-03, 8.2422e-03, 7.5573e-03, 7.1725e-03, 6.8706e-03,
        5.9860e-03, 5.7899e-03, 5.2757e-03, 4.6894e-03, 4.5944e-03, 4.3452e-03,
        4.3294e-03, 4.2805e-03, 4.1815e-03, 4.1676e-03, 4.0946e-03, 3.9254e-03,
        3.6068e-03, 3.2053e-03, 3.1196e-03, 3.0914e-03, 2.9854e-03, 2.8490e-03],
       device='cuda:0')

tensor([ 7163, 27252,     3,    59, 10747,  1465,  2841,  1877,     6,    96,
           16, 10998,   220,   673,  6864,    20,   105, 32097,     5,   204,
         4486,   209,     1, 32095, 32081, 19197,    18, 32093,   126, 32078],
       device='cuda:0')

In [ ]:
# tokens, that have the hightest predictability for entailment, using the prompt hypothesis: and premise:

tensor([2.9324e+03, 3.4392e+02, 1.3448e+02, 9.1153e-02, 3.6827e-02, 3.3527e-02,
        3.1646e-02, 2.5204e-02, 2.0629e-02, 1.9164e-02, 1.7800e-02, 1.5784e-02,
        1.3746e-02, 1.3442e-02, 1.3244e-02, 1.3197e-02, 1.2865e-02, 1.1811e-02,
        1.1616e-02, 1.1422e-02, 9.4241e-03, 8.9763e-03, 8.6653e-03, 8.1657e-03,
        8.0583e-03, 7.8613e-03, 7.8076e-03, 7.7779e-03, 7.3928e-03, 7.1338e-03],
       device='cuda:0')
tensor([    3,  7163, 27252,    59, 10998,  1877,     6, 19197,  6864,    16,
            5,    96,   220,    20,   673,     1, 32097, 32095, 10747, 32081,
          105, 32078, 32093,   204,    18, 12681,  1465,  8724, 32080,   209],
       device='cuda:0')

In [ ]:
#using labels 10998/1465/206 we get acc of 0.835(noramlizer, without normalization its 0.825)

# mit 19197/1465/206 sogar acc von 0.8425 (habe ich so ausgewählt, dass es hohe wahrscheinlichkeit für entailment hat, aber niedrige für contradiction/neutral)

# mit ""/neutral/contradiction bekommt man acc von 0.8625

# Prediction token ensemble
multiple tokens are defined correspond to a class. Sum over the prob of those tokens and then choose the label, for which the ensemble has the highest prob.

In [ ]:
#TODO nicht einfach die token zusammenzählen, sondern für jeden token aussrechnen, wie stark er zu einem label tendiert. Die token, die dnan einen hohen bias haben, sollte man als label nehmen

0.16666666666666666

In [ ]:
#die summe hängt davon ab, wie viele labels es im datensatz gab-> also TODO durch Anzahl teilen

token_indices = [3,7163,27252,59,10998,1877]
ent_probs = np.array([2.9324e+03,3.4392e+02,1.3448e+02,9.1153e-02,3.6827e-02,3.3527e-02])
neutral_probs = np.array([2.3008e+02,2.4199e+03,2.5138e+02,8.5957e-02,6.8706e-03,8.6295e-03])
contr_probs = np.array([ 7.0882e+01,2.4327e+02,3.3714e+03,9.6478e-02,3.0029e-03,4.2940e-03])

sum = ent_probs + neutral_probs + contr_probs

for i, element in enumerate(sum):
  ent_probs[i] = ent_probs[i] / sum[i]
  neutral_probs[i] = neutral_probs[i] / sum[i]
  contr_probs[i] = contr_probs[i] / sum[i]

print(ent_probs)
print(neutral_probs)
print(contr_probs)

from tqdm import tqdm_notebook
import numpy as np
import torch.nn.functional as F
num_right = 0


ent_amount = 0
with torch.no_grad():
  for i, mnli_instance in enumerate(tqdm_notebook(testloader)):

      text = "hypothesis: " + mnli_instance["hypothesis"][0] + " premise: " + mnli_instance["premise"][0]
      input_ids = tokenizer.encode(text,return_tensors="pt").cuda()
      logits = F.softmax(model(input_ids=input_ids,decoder_input_ids =torch.tensor([[0]]).cuda()).logits[:,-1,:])
      probs = torch.cat([logits[:,3]/sum[0], logits[:,7163]/sum[1],logits[:,27252]/sum[2],logits[:,59]/sum[3],logits[:,10998]/sum[4],logits[:,1877]/sum[5]]).cpu().numpy()

      label = np.argmax([np.sum(ent_probs*probs), np.sum(neutral_probs*probs),np.sum(contr_probs*probs)])
      
      if label == mnli_instance["label"]:
          num_right+=1

num_right/1000

[0.90691979 0.11436971 0.03579204 0.33317616 0.78857828 0.7217791 ]
[0.07115813 0.80473148 0.06690514 0.3141841  0.14712048 0.18577841]
[0.02192207 0.08089881 0.89730282 0.35263974 0.06430124 0.09244249]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0.91

In [ ]:
num_right/1000

0.91

# New Section

In [ ]:
from tqdm import tqdm_notebook
import numpy as np
import torch.nn.functional as F
num_right = 0
ent_count = 0
contr_count = 0
neutral_count = 0

ent_amount = 0
with torch.no_grad():
  for i, mnli_instance in enumerate(tqdm_notebook(testloader)):
      #if i>10:
      #    break
      text = "Consider the following hypothesis: " + mnli_instance["hypothesis"][0] + " premise: " + mnli_instance["premise"][0]
      input_ids = tokenizer.encode(text,return_tensors="pt").cuda()
      logits = F.softmax(model(input_ids=input_ids,decoder_input_ids =torch.tensor([[0]]).cuda()).logits[:,-1,:])
      label = torch.argmax(torch.cat([logits[:,3]/304, logits[:,7163]/327,logits[:,27252]/367]))
      ent_count+=logits[:,3].item()
      neutral_count+=logits[:,7163].item()
      contr_count+=logits[:,27252].item()
      if label==0:
        #print(logits[:,3].item())
        #print(logits[:,7163].item())
        #print(logits[:,27252].item())
        #print()
        ent_amount+=1
      #if logits[:,28722] > 0 or logits[:,7163] > 0 or logits[:,27252] > 0:
      #  print("trulll")
      
      if label.cpu() == mnli_instance["label"]:
          num_right+=1
      #else:
      #  print(text)
      #  print(label)
      #  print(mnli_instance["label"])

num_right/test_length



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


0.851

In [ ]:
print(ent_count)
print(neutral_count)
print(contr_count)

304.84313232914565
327.3357807310931
367.56872040954477


In [ ]:
tokenizer.encode(" entailment")

[3, 35, 5756, 297, 1]

In [ ]:
text = ["hypothesis: lala premise:lalal","xd"]
input_ids = tokenizer(text,return_tensors="pt",padding=True).input_ids.cuda()
model.generate(input_ids=input_ids)

tensor([[    0,     3,    35,  5756,   297,     1,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [    0, 32099,     3,   226,    26,     3,   226,    26,     3,   226,
            26,     3,   226,    26,     3,   226,    26,     3,   226,    26]],
       device='cuda:0')

# Batch predicition string comparision

In [ ]:
#batches
from tqdm import tqdm_notebook
import numpy as np
import torch.nn.functional as F

num_right = 0
ent_count = 0
contr_count = 0
neutral_count = 0

ent_amount = 0

ent = [0, 3, 35, 5756, 297, 1]
neutral = [0, 7163, 1,0,0,0]
contr = [0, 27252, 1,0,0,0]
with torch.no_grad():
  for i, mnli_instance in enumerate(tqdm_notebook(testloader)):

      text = ["mnli hypothesis:" + mnli_instance["sentence1"][j] + " premise:" + mnli_instance["sentence2"][j] for j in range(0,len(mnli_instance["sentence1"]))]


      input_ids = tokenizer(text,return_tensors="pt",padding=True).input_ids.cuda()

      output = model.generate(input_ids,num_beams=2)




      labels = [1 if list(tokens.cpu().numpy())==ent else 0 if list(tokens.cpu().numpy())==neutral else 0 if list(tokens.cpu().numpy())==contr else -1 for tokens in output]

      num_right+= torch.sum(torch.tensor(labels) == mnli_instance["label"])





num_right/test_length


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


tensor(0.3200)

In [ ]:
print(tokenizer.decode(output[0]))

<pad><extra_id_0> Everyone really likes the newest benefits The new rights are nice enough The new rights are


In [ ]:
tokenizer.encode("pre")

[554, 1]

In [ ]:
tokenizer.decode([    0, 32099,    37])

'<pad><extra_id_0> The'

In [ ]:
from tqdm import tqdm_notebook
#TDOD change again to premise, hypothesis instead of the other way around
accuracies = dict.fromkeys(prompts[0] , 0)
accuracies = {x: dict.fromkeys(prompts[1] , 0) for x in accuracies}
for key,value in accuracies.items():
  accuracies[key] =  {x: dict.fromkeys(prompts[2] , 0) for x in value}
for key,value in accuracies.items():
  for key2,value2 in value.items():
    accuracies[key][key2] =  {x: dict.fromkeys(prompts[3] , 0) for x in value2}

len_ent = 0
len_neu = 0
len_contr = 0

counter = 0 

for i, mnli_instance in enumerate(tqdm_notebook(dataloader_mnli)):
      for q,trigger in enumerate(prompts[0]):
        for j,X in enumerate(prompts[1]):
          for k,Y in enumerate(prompts[2]):
            for l,Z in enumerate(prompts[3]):
              hypothesis = mnli_instance["hypothesis"][0]
            

              premise = mnli_instance["premise"][0]

              
              
              counter+=1
              text = trigger + X + "Humans are basketballs. Or  "+  hypothesis + Y + premise + Z
              input_ids = tokenizer.encode(text,return_tensors="pt").cuda()
              label = tokenizer.decode(model.generate(input_ids,num_beams=3)[0],skip_special_tokens=True)
              if label == "entailment":
                len_ent +=len(hypothesis)/len(premise) 
                label = 0
              elif label == "neutral":
                len_neu+= len(hypothesis)/len(premise) 
                label = 1
              elif label == "contradiction":
                len_contr+=len(hypothesis)/len(premise) 
                label = 2

              if label == mnli_instance["label"] :#or (label==1 and mnli_instance["label"]==0)
                accuracies[trigger][X][Y][Z]+=1



accuracies

#dict statt list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


{'': {'Consider the following hypothesis: ': {' premise: ': {'': 212}}}}

In [ ]:
print(len_ent/counter)
print(len_neu/counter)
print(len_contr/counter)

0.0
0.41877066608358887
0.24863980975965547


In [ ]:
import copy
accuracies_p = copy.deepcopy(accuracies)

for key0,value0 in accuracies_p.items():
  for key1,value1 in value0.items() :
    for key2,value2 in value1.items() :
      for key3,value3 in value2.items():
        accuracies_p[key0][key1][key2][key3] = value3/counter#mnli_len

accuracies_p



{'': {'Consider the following hypothesis: ': {' premise: ': {'': 0.53}}}}

# Batch logits


In [ ]:
tokenizer.encode("contradiction")

[27252, 1]

In [ ]:
#batches
from tqdm import tqdm_notebook
import numpy as np
import torch.nn.functional as F

num_right = 0

ent_count = 0
contr_count = 0
neutral_count = 0

ent_amount = 0


with torch.no_grad():
    for i, mnli_instance in enumerate(tqdm_notebook(testloader)):

        text = ["mnli hypothesis:" + mnli_instance["sentence1"][j] + " premise:" + mnli_instance["sentence2"][j] for j in range(0,len(mnli_instance["sentence1"]))]



        input = tokenizer(text,return_tensors="pt",padding=True)

        probs = F.softmax(model(input_ids=input["input_ids"].cuda(),attention_mask=input["attention_mask"].cuda(),decoder_input_ids =torch.tensor([[0]]*len(text)).cuda()).logits[:,-1,:])
  
        labels = torch.argmax(torch.cat([probs[None,:,3]/1,probs[None,:,7163]/1,probs[None,:,27252]/1]),dim=0)

    
        num_right+= torch.sum(labels.cpu() == mnli_instance["label"])
        
        ent_count+= torch.sum(probs[None,:,22945])
        neutral_count+= torch.sum(probs[None,:,1427])
        contr_count+= torch.sum(probs[None,:,632])
  
        




      




print(num_right/test_length)
print(ent_count)
print(neutral_count)
print(contr_count)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



tensor(0.5028)
tensor(0.0017, device='cuda:0')
tensor(0.0117, device='cuda:0')
tensor(0.0259, device='cuda:0')


In [ ]:
0.3476

In [ ]:
tokenizer.decode(632)

'0'

# Results



1.   Ohne die trigger premise und hypothesis ist acc random
2.   Teilweise haben label eine hohe acc die intuitiv sind. (zb label 1/2/3 vertauscht zu 1/3/2)
3.   Man muss die wahrscheinlichkeiten normalisieren bevor man predicted, also teilen durch die wkeitet, dass dieses token im durchschnitt im trainingsset hat(sonst würde zb bei label 1/neutral/2 immer neutral ausgegebene werden)
4.   Es gibt andere label die eine hohe acc erzielen:
Eines der label kann man frei wählen, ohne dass acc viel weniger wird
good=entailment, bad=contradiction hat auch hohe acc, obwohl die labels im strengen Sinne nichts miteinernader zu tun haben

In [ ]:
"conclusion: " + mnli_instance["hypothesis"][0] + "assumption: " + mnli_instance["premise"][0]
ohne normalizing 52% acc, mit jedoch nur 41%

In [ ]:
"" + mnli_instance["hypothesis"][0] + " " + mnli_instance["premise"][0]

auch normalisiert nur 35% acc

In [ ]:
interessant, dass ich good=entailment und bad=contradiction gewählt habe, obwohl ja eigentlich nichts inhärent schlechtes an einer contradiciton ist, ist es für mich trd negativ annotiert
synoyme oder solche intuitionen für neutral zu finden ist viel schwieriger

In [ ]:
"hypothesis: " + mnli_instance["hypothesis"][0] + " premise: " + mnli_instance["premise"][0]

mit label implication/neutral/contradiction hat 80% acc normalized

aber mit banana/neutral/contradiction hat es 83% normalized (also wenn sich t5 sicher ist dass es 1 oder 2 ist macht es eines von denen, sonst 0)

denn mit banana/apple/contradiciton ist acc bei 0.4 normalized

mit implication/neutral/conflict bekommt man 0.35

mit implication/banana/conflict bekommt man 0.24, also besser als random nur falsch rum?

mit good/neutral/bad bekommt man acc von 79%, normalized!!!

mit good/banana/bad bekommt man 51%

mit great/neutral/horrible bekommt man 65%

mit great/strawberry/horrible bekommt man 52%

mit cute/neutral/ugly bekommt man 45%

mi cute/whatever/ugly bekommt man 22%,also besser als random aber falsch rum

mit good/whatever/bad bekommt man 55%

In [ ]:
"question: Which label best describes the relationship between hypothesis and premise? context: hypothesis: " + mnli_instance["hypothesis"][0] + " premise" + mnli_instance["premise"][0] + "label 1: entailment, label 2 = neutral, label 3 = contradiction"

mit label 1/2/3 gibt 40% acc, schlechter wenn nicht normalisiert

In [ ]:
"question: Which label best describes the relationship between statement 1 and statement 2? context: statement 1: " + mnli_instance["hypothesis"][0] + " statement 2:" + mnli_instance["premise"][0] + "label 1: entailment, label 2 = neutral, label 3 = contradiction"

label ""/neutral/contradiction auch normalisiert nur 36%
label 1/2/3 auch normalisiert nur 32%

In [ ]:
"mnli hypothesis:" + mnli_instance["hypothesis"][0] + " premise:" + mnli_instance["premise"][0] also ganz normal mnli

wenn man hier als label yes/no/maybe verwendet und bei einer prediction jeweils die wkeit durch die gesammtwkeit, dass dieses label kommt teil, bekommt man 78%

man bekommt nur random acc wenn man nicht normalisiert

das selbe mit apple/banana/strawberry bekommt 42%, also etwas besser als random?

In [ ]:
"question: Does the premise entail the hypothesis? context: hypothesis:" + mnli_instance["hypothesis"][0] + " premise: " + mnli_instance["premise"][0] 
mit label yes(entailment) und no(neutral und contradiction) hat acc von 65%

funktioniert auch, wenn man : bei hypotesis weglässt

zeigt aber noch nicht viel, da man 66% bekommt, wenn man einfach immer no ausgibt und genau das macht t5 hier auch

wenn man aber yes/16 und no/13 teilt(uncond prob), so bekommt man 68% acc und es wird 411 von 1000 mal yes ausgegeben

in diesem fall hilt auch hypothesis: ein wenig, dann bekommt man acc von 73%

aber hypothesis und premise ganz weglassen führt zu acc von 50%

die gesammte frage und kontext weglassen, aber hypothesis und premise stehenlassen hat acc 71%

In [ ]:
"question: Which of the following options best describe the relationship between them? 1. The premise entails the hypothesis. 2. The premise is neutral to the hypothesis. 3. The premise contradicts the hypothesis. context: hypothesis:" + mnli_instance["hypothesis"][0] + " premise: " + mnli_instance["premise"][0] 

gibt random acc

In [ ]:
"Consider the following hypothesis: " + mnli_instance["hypothesis"][0] + " Consider the following premise: " + mnli_instance["premise"][0] + "Which of the following options best describe the relationship between them? 1. The premise entails the hypothesis. 2. The premise is neutral to the hypothesis. 3. The premise contradicts the hypothesis. "

gibt 16% acc, wenn man als label 1,2,3 nimmt. Wenn man jedoch 1 und 3 label vertauscht bekommt man eine acc von 57%. Also hat T5 darüber etwas gelernt, aber das falsche?

kann natürlich sein, dass dataset imbaleneced ist, aber es zeigt sich auch wenn man statt 400 2000 instances nimmt

wenn  man triggertoken "hypothesis:" weglässt, bekommt man random acc

In [ ]:
TODO: schauen ob aktivierung bei T5 im mnli fall die selben sind wie aktivierungen im qa fall